In [32]:
import importlib
import os
import gwflow_creation
importlib.reload(gwflow_creation)
from gwflow_creation import create_gwflow

LEVEL = "huc12"
RESOLUTION = 250
BASE_PATH = fr"/data/MyDataBase/SWATGenXAppData/"
NAMES = os.listdir("D:\MyDataBase\SWATplus_by_VPUID\huc12")
NAME = '40900010209'
CRS = 26990
create_gwflow(NAME,RESOLUTION, LEVEL, BASE_PATH,CRS)
os.chdir(fr"D:\MyDataBase\SWATplus_by_VPUID\huc12\{NAME}\gwflow")
!gwflow_input_files.exe

#!swat+gwflow_Ryan_version.exe

begin
Target path: /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40900010209/gwflow
gis_folder /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40900010209/gwflow\gwflow_gis
Performing ArcMap routines to set up gwflow cell data...
     create watershed boundary
using existing grid shapefile
fishnet are generated based on existing gridshapefile
     identify cells within watershed boundary
          clip grid to watershed area
          perform spatial join
     determine aquifer thickness for each grid cell
          convert raster to polygon
columns: Index(['Id', 'active', 'geometry', 'thick_m'], dtype='object')
thick_m volumn: [4.8768 4.8768 4.8768 ... 3.6576 3.3528 3.6576]
Geometry columns: geometry
Number of geometry columns: 1
     determine ground surface elevation for each grid cell
          perform spatial join
     determine aquifer properties for each grid cell
          clip global dataset to watershed area
          calculate K (m/day)
          assign a

forrtl: severe (408): fort: (10): Subscript #1 of the array HRU_CONNECTED has value 13729 which is greater than the upper bound of 13728

Image              PC                Routine            Line        Source             
gwflow_input_file  00007FF68D005961  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68CFE345C  MAIN__                    415  gwflow_input_files.f
gwflow_input_file  00007FF68D09D052  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68D09E329  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68D09E24E  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68D09E10E  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68D09E399  Unknown               Unknown  Unknown
KERNEL32.DLL       00007FFB5BDE4DE0  Unknown               Unknown  Unknown
ntdll.dll          00007FFB5C0FED9B  Unknown               Unknown  Unknown


In [7]:
import arcpy
import geopandas as gpd
import numpy as np
import pandas as pd
import os
def read_grids(grids_polygon_path, CRS):
    
    temp_grids = pd.read_pickle(grids_polygon_path)
    temp_grids ['Cell_ID'] = np.arange(1, len(temp_grids)+1)
    
    grids_polygon=  gpd.GeoDataFrame(temp_grids, crs=CRS, geometry='geometry').copy()
    
    temp_grids['centroid'] = temp_grids.geometry.centroid
    temp_grids.drop(columns='geometry', inplace=True)
    temp_grids.rename(columns={'centroid': 'geometry'}, inplace=True)
    grids_point = gpd.GeoDataFrame(temp_grids, geometry='geometry', crs=CRS)
    grids_point_path= os.path.join(gis_folder, 'grids_points.shp')
    grids_point.to_file(grids_point_path)
    
    return grids_polygon, grids_point_path

def clear_directory(directory_path):
    # Check if the directory exists
    if not os.path.exists(directory_path):
        print("Directory does not exist:", directory_path)
        return
    # Iterate over each file and directory inside the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
            
def extract_raster_values_to_points(point_shapefile, raster_path, output_shapefile, gis_folder, column_name):
    arcpy.env.workspace = gis_folder
    arcpy.env.overwriteOutput = True

    # Perform the extraction
    temp_output = "in_memory/temp_output"
    arcpy.sa.ExtractValuesToPoints(point_shapefile, raster_path, temp_output)
    arcpy.AddField_management(temp_output, column_name, "FLOAT")
    arcpy.CalculateField_management(temp_output, column_name, "!RASTERVALU!", "PYTHON3")
    arcpy.CopyFeatures_management(temp_output, output_shapefile)
    arcpy.DeleteField_management(in_table=output_shapefile, drop_field="RASTERVALU")
    print(f"Extracted values saved to: {output_shapefile}")
    outpath = os.path.join(gis_folder, output_shapefile)
    return outpath


def defining_bound_and_active(subbasin_path, gis_folder):
    Subbasin = gpd.read_file(subbasin_path)
    active_domain = Subbasin.dissolve().reset_index(drop=True)
    active_domain = active_domain.set_geometry('geometry').copy()
    bound = active_domain.boundary.copy()
    bound = bound.explode(index_parts=False)
    bound = bound[bound.length == bound.length.max()]
    # Optional: bound = bound.buffer(RESOLUTION)
    
    # Ensure 'bound' is a GeoDataFrame
    if not isinstance(bound, gpd.GeoDataFrame):
        bound = gpd.GeoDataFrame(geometry=bound)

    # Set CRS for 'bound' if it's not already set
    if not bound.crs:
        bound.crs = active_domain.crs

    bound['Bound'] = 1
    active_domain['Active'] = 1

    bound_path = os.path.join(gis_folder, 'bound_shape.shp')
    active_domain_path = os.path.join(gis_folder, 'active_domain_shape.shp')

    bound[['Bound','geometry']].to_file(bound_path)
    print('Generated shap saved to:',os.path.basename(bound_path))
    active_domain[['Active','geometry']].to_file(active_domain_path)
    print('Generated shape saved to:',os.path.basename(active_domain_path))
    
    return bound_path, active_domain_path

def getting_TABLES(outpath_object):
    
    subbasin_path = outpath_object['Subbasin']
    hru_path = outpath_object['HRUS']
    channel_path = outpath_object['Channel']
    elev_path = outpath_object['Elev']
    Zone_path = outpath_object['Zone']
    
    HRUs = gpd.GeoDataFrame(pd.read_pickle(hru_path), geometry='geometry')
    DEM = gpd.read_file(elev_path)
    Zone = gpd.read_file(Zone_path)
    Channels = gpd.GeoDataFrame(pd.read_pickle(channel_path), geometry='geometry')
    Subbasin = gpd.GeoDataFrame(pd.read_pickle(subbasin_path), geometry='geometry')
    active_cells = Subbasin.Cell_ID.unique()
  
    HRUs = HRUs.assign(poly_area_m2=HRUs.geometry.area, Cell_Area_m2=RESOLUTION*RESOLUTION).drop(columns='geometry')

    HRUs['HRUs_Area_m2'] = HRUs['HRUs_Area_m2'].astype(float).round(2)
    HRUs['Cell_Area_m2'] = HRUs['Area_m2'].astype(float).round(2)
    HRUs['poly_area_m2'] = HRUs['poly_area_m2'].astype(float).round(2)
    HRUs['Cell_ID'] = HRUs['Cell_ID'].astype(int)
    HRUs['HRUS'] = HRUs['HRUS'].astype(int)  

    HRU_CELL_TABLE = HRUs.sort_values(by=['Cell_ID','HRUS'], ascending=True).reset_index(drop=True)[['Cell_ID', 'HRUs_Area_m2', 'HRUS', 'poly_area_m2']] ### sorted first by CELL number then by HRU    ['Cell_ID','Area_m2','HRU','poly_area_m2']
    CEll_HRU_TABLE = HRUs.sort_values(by=['HRUS', 'Cell_ID'], ascending=True).reset_index(drop=True)[['Cell_ID', 'Cell_Area_m2', 'HRUS', 'poly_area_m2']]  #### sorted first by cell_id and then by HRU number  ['Cell_ID','Area_m2', 'HRU', 'poly_area_m2']
      

    
    
    ## preparing channel table
    Channels = Channels.merge(DEM[['Elev','Cell_ID']], on='Cell_ID', how='left')
    Channels = Channels.merge(Zone[['Zone','Cell_ID']], on='Cell_ID', how='left')
    Channels = Channels.assign(Length=Channels.geometry.length).drop(columns='geometry')
    
    CELL_CHANNEL_TABLE = Channels.sort_values(by=['Cell_ID'], ascending=True).reset_index(drop=True)[['Cell_ID'	 ,'Channel' ,'Zone'  , 'Elev'   , 'Length']]
    
    CELL_CHANNEL_TABLE['Elev'] = CELL_CHANNEL_TABLE['Elev'].astype(float).round(2)
    CELL_CHANNEL_TABLE['Length'] = CELL_CHANNEL_TABLE['Length'].astype(float).round(2)
    CELL_CHANNEL_TABLE['Cell_ID'] = CELL_CHANNEL_TABLE['Cell_ID'].astype(int)
    CELL_CHANNEL_TABLE['Zone'] = CELL_CHANNEL_TABLE['Zone'].astype(int)
    CELL_CHANNEL_TABLE['Channel'] = CELL_CHANNEL_TABLE['Channel'].astype(int)

    print('Length of channels_cell array:',len(Channels))
    print('Maximum number of Channels:', Channels['Channel'].max())
    print('Length of CELL_HRUs array:',len(HRUs))
    print('Maximum number of HRUs:', HRUs['HRUS'].max())
    

    return CEll_HRU_TABLE, HRU_CELL_TABLE, CELL_CHANNEL_TABLE


def getting_grid_table(outpath_object):

    feet_to_meter=0.3048
    Leke_bed_depth=2
    Lake_stage_height=2
    
    subbasin_path       = outpath_object['Subbasin']
    hru_path            = outpath_object['HRUS']
    channel_path        = outpath_object['Channel']
    elev_path           = outpath_object['Elev']
    zone_path           = outpath_object['Zone']
    lakes_path          = outpath_object['LakeId']
    Thick_path          = outpath_object['Thick']
    SWL_path            = outpath_object['SWL']
    Bound_path          = outpath_object['Bound']
    Active_path         = outpath_object['Active']
    grids_point_path    = outpath_object['grids_point']
    grids_polygon_path  = outpath_object['grids_polygon']
    
    grids_point = gpd.read_file(grids_point_path)
    print('check:','Number of grid cells loaded', len(grids_point))
    
    Thick  = gpd.read_file(Thick_path)
    SWL    = gpd.read_file(SWL_path)
    DEM    = gpd.read_file(elev_path)
    Zone   = gpd.read_file(zone_path)
    
    Lakes    = gpd.GeoDataFrame(pd.read_pickle(lakes_path), geometry='geometry').fillna(0)
    HRUs     = gpd.GeoDataFrame(pd.read_pickle(hru_path), geometry='geometry')
    Channels = gpd.GeoDataFrame(pd.read_pickle(channel_path), geometry='geometry')
    Subbasin = gpd.GeoDataFrame(pd.read_pickle(subbasin_path), geometry='geometry')
    Bound    = gpd.GeoDataFrame(pd.read_pickle(Bound_path), geometry='geometry').drop_duplicates(subset='Cell_ID').fillna(0)
    Active   = gpd.GeoDataFrame(pd.read_pickle(Active_path), geometry='geometry').drop_duplicates(subset='Cell_ID').fillna(0)
    
    
    ### preparing Grid table
    grids_point = grids_point.merge(Bound[['Bound','Cell_ID']], on='Cell_ID', how='left')
    grids_point = grids_point.merge(Active[['Active','Cell_ID']], on='Cell_ID', how='left')
    grids_point = grids_point.merge(DEM[['Elev','Cell_ID']], on='Cell_ID', how='left')
    grids_point = grids_point.merge(Thick[['Thick','Cell_ID']], on='Cell_ID', how='left')
    grids_point = grids_point.merge(Zone[['Zone','Cell_ID']], on='Cell_ID', how='left')
    grids_point = grids_point.merge(Lakes[['LakeId','Cell_ID']], on='Cell_ID', how='left').rename(columns={"LakeId":"Lake"})
    grids_point = grids_point.merge(SWL[['SWL','Cell_ID']], on='Cell_ID', how='left')
    grids_point = grids_point.fillna(0)

    #### defining lakes
    lake_index = grids_point[grids_point['Lake']==1].index
    grids_point ['Lake_bed'] = 0
    grids_point ['Lake_stage'] = 0
    grids_point.loc[lake_index, 'Lake_bed']   = grids_point.loc[lake_index, 'Elev'] - Leke_bed_depth
    grids_point.loc[lake_index, 'Lake_stage'] = grids_point.loc[lake_index, 'Elev'] + Lake_stage_height

    #### defining heads
    grids_point ['Head']       = grids_point['Elev'] - feet_to_meter*grids_point['SWL']   
    grids_point.loc[lake_index, 'Head'] = grids_point.loc[lake_index, 'Elev']

    ### nutrient & other parameters
    grids_point ['InitP']      = 0
    grids_point ['InitNO3']    = 0
    grids_point ['Tile']       = 0
    grids_point ['ET_fix']     = 0
    grids_point ['EXDP']       = 0

    num_rows = grids_point.Row.max()
    num_cols = grids_point.Col.max()
    
    GRIDS_TABLE = grids_point
    print('check:','Total number of grids cell', len(GRIDS_TABLE))
    GRIDS_TABLE_path = os.path.join(gis_folder,'grids_table.txt')
    GRIDS_TABLE.to_csv(GRIDS_TABLE_path)
    int_columns= ['Cell_ID',"Active",'Bound','Lake','Zone']

    float_columns= ['Thick',"Elev",'Head','Lake_bed','Lake_stage','EXDP',"ET_fix",'Tile','InitNO3','InitP']
    
    GRIDS_TABLE.loc[:, int_columns] = GRIDS_TABLE[int_columns].astype(int)

    GRIDS_TABLE.loc[:, float_columns] = GRIDS_TABLE[float_columns].astype(float).round(2)


    GRIDS_TABLE = GRIDS_TABLE[[
        
               "Cell_ID", "Active", "Thick", "Elev",
                    "Zone","Bound", "Head", "EXDP", "ET_fix",
                    "Tile","InitNO3", "InitP","Lake",
                        "Lake_bed","Lake_stage"
                ]]
    return grids_point ,GRIDS_TABLE,  GRIDS_TABLE_path, num_rows, num_cols


def creating_zones(path_to_HHC, feet_to_meter=0.306):
    HHC = gpd.read_file(path_to_HHC)
    
    # Convert feet to meter in HHC and handle NaN values
    HHC['HHC'] = HHC['HHC'].apply(lambda x: x * feet_to_meter if np.isfinite(x) else np.nan)
    
    # Calculate quantiles excluding NaN values
    first_quartile = np.nanpercentile(HHC.HHC.dropna(), 25)
    median = np.nanpercentile(HHC.HHC.dropna(), 50)
    third_quartile = np.nanpercentile(HHC.HHC.dropna(), 75)

    # Print quantiles for verification
    print('first_quartile:', first_quartile)
    print('median:', median)
    print('third_quartile:', third_quartile)

    # Create a copy to avoid SettingWithCopyWarning
    Zone = HHC[['Cell_ID', 'HHC', 'geometry']].copy()

    # Initialize 'Zone' column and assign values considering NaN
    Zone['Zone'] = np.select(
        [
            Zone['HHC'].isna(), 
            Zone['HHC'] <= first_quartile, 
            (Zone['HHC'] > first_quartile) & (Zone['HHC'] <= median),
            (Zone['HHC'] > median) & (Zone['HHC'] <= third_quartile),
            Zone['HHC'] > third_quartile
        ], 
        [
            np.nan, 1, 2, 3, 4
        ],
        default=np.nan
    )

    # Set 'Sy' values incrementally for each zone
    for zone_number in range(1, 5):
        increment = zone_number * 0.05
        Zone.loc[Zone['Zone'] == zone_number, 'Sy'] = increment

    # Calculate representative value (mean) for each zone
    for zone_number in range(1, 5):
        zone_HHC_values = Zone[Zone['Zone'] == zone_number]['HHC']
        k = np.nanmean(zone_HHC_values)
        Zone.loc[Zone['Zone'] == zone_number, 'k'] = k

    outpath = os.path.join(gis_folder, 'Zone_Shape.shp')
    Zone.to_file(outpath)
    print('Zones are created and saved:', 'Zone_Shape.shp')

    
    Zone['n'] = 0.3
    
    ZONE_K_TABLE = Zone[['Zone', 'k', 'Sy', 'n']]
    ZONE_K_TABLE = ZONE_K_TABLE.groupby('Zone').mean().reset_index(drop=False)
    
    ZONE_K_TABLE['Zone'] = ZONE_K_TABLE['Zone'].astype(int)
    ZONE_K_TABLE['k'] = ZONE_K_TABLE['k'].astype(float).round(2)
    ZONE_K_TABLE['Sy'] = ZONE_K_TABLE['Sy'].astype(float).round(2)
    ZONE_K_TABLE['n'] = ZONE_K_TABLE['n'].astype(float).round(2)

    
    return Zone, ZONE_K_TABLE, outpath


import os
def write_table(table, file):
    try:
        table.to_csv(file, sep='\t', header=False, index=False, line_terminator='\n')
    except Exception as e:
        print(f"Error writing table: {e}")

def write_gwflow_file(gwflow_target_path, ZONE_K_TABLE, GRIDS_TABLE, CELL_CHANNEL_TABLE, HRU_CELL_TABLE, CEll_HRU_TABLE,num_rows, num_cols):
    gwflow_data_path = os.path.join(gwflow_target_path, 'gwflow.data')

    # Clear or create the output file
    with open(gwflow_data_path, 'w') as file:
        # Writing static text
        file.write("This file contains Vahid's data that will be processed to create the gwflow input files\n\n")
        file.write("Basic information\n")
        file.write("500\n")
        file.write(f"{num_rows+1} {num_cols+1}\n")
        file.write("3\t\t\tWater table initiation flag\n")
        file.write("10.0\n")
        file.write("1\t\t\tBoundary conditions (1=constant head; 2=no flow)\n")
        file.write("1\t\t\tGroundwater-->soil transfer is simulated (0 = no; 1 = yes)\n")
        file.write("1\t\t\tSaturation excess flow is simulated (0 = no; 1 = yes)\n")
        file.write("1\t\t\tGroundwater ET is simulated (0 = no; 1 = yes)\n")
        file.write("1\t\t\tTile drain flow is simulated (0 = no; 1 = yes)\n")
        file.write("1\t\t\tGroundwater-Lake interaction is simulated (0 = no; 1 = yes)\n")
        file.write("0\t\t\tGroundwater pumping is specified by user (0 = no; 1 = yes)\n")
        file.write("1\t\t\tRecharge is passed from HRUs (0 = no; 1 = yes)\n")
        file.write("3.0\n")
        file.write("0\t\t\tGroundwater nutrient transport (0 = no; 1 = yes)\n")
        file.write("1.0\n")
        file.write("1 1 1\t\t\tFlags for groundwater and nutrient mass balance files (daily; annual; average annual)\n")
        file.write("\n")
        file.write("Aquifer Properties, by zone\n")
        file.write(f"{len(Zone.Zone.unique())}\n")
        file.write("Zone\tK (m/d)\tSy\tn\n")

        # Write dynamic table
 
        write_table(ZONE_K_TABLE, file)

        file.write("\n")
        file.write("Streambed Properties, by zone\n")
        file.write("1\tNumber of zones\n")
        file.write("Zone K (m/d) Thickness (m)\n")
        file.write("1\n")
        file.write("0.005\n")
        file.write("1.00\n")
        file.write("\n")
        file.write("Groundwater Head Output Control (days for which head will be output for each grid cell)\n")
        file.write("1  Number of output times\n")
        file.write("2010 270\n")
        file.write("\n")
        file.write("Groundwater Head Observation Locations (cells for which head will be output for each time step)\n")
        file.write("0\n")
        file.write("\n")
        file.write("Cell for detailed daily sources/sink output\n")
        file.write("Row\tColumn\n")
        file.write("1\t1\n")
        file.write("\n")
        file.write("River Cell Information\n")
        file.write("5.0\n")
        file.write("\n")
        file.write("Hydrograph separation\n")
        file.write("1\t\t\tNumber of channels\n")
        file.write("1\t\t\tChannel ID\n")
        file.write("\n")
        file.write("Tile Drain Parameters\n")
        file.write("1.22\n")
        file.write("50.0\n")
        file.write("5.00\n")
        file.write("0\t\t\tTile cell groups (flag: 0=no; 1=yes)\n")
        file.write("\n")
        file.write("Lake Parameters\n")
        file.write("2.00\n")
        file.write("0.005\n")
        file.write("2.00\t\t\tNO3-N concentration (mg/L) of lake water\n")
        file.write("0.01\t\t\tP concentration (mg/L) of lake water\n")
        file.write("\n")
        file.write("Groundwater nutrient transport\n")
        file.write("-0.0001\t\t\tFirst-order rate constant for denitrification (1/day)\n")
        file.write("5.00\t\t\tDispersion coefficient (m2/day)\n")
        file.write("1.00\t\t\tSorption retardation coefficient for Nitrate\n")
        file.write("2.00\t\t\tSorption retardation coefficient for Phosphorus\n")
        file.write("1\t\t\tNumber of transport time steps per flow time step\n")
        file.write("\n")
        file.write("Cell Information (sorted by Cell ID)\n")
        file.write("Cell_ID\tActive\tThick\tElev\tZone\tBound\tHead\tEXDP\tET_fix\tTile\tInitNO3\tInitP\tLake\tLake_bed\tLake_stage\n")

        # Write dynamic table
        write_table(GRIDS_TABLE, file)
        file.write("\n")
        file.write("Channel-Cell Connection Information\n")
        file.write(f"{len(CELL_CHANNEL_TABLE)} Number of cells that intersect channels\n")
        file.write("CellID\tChannel\tZone\tElev\tLength\n")

        # Write dynamic table
        write_table(CELL_CHANNEL_TABLE, file)
        file.write("\n")
        file.write('HRU-Cells Connection Information\n')              
        file.write(f'{len(CEll_HRU_TABLE)} Number of connections\n')
        file.write("Cell_ID\tArea_m2\tHRU\tpoly_area_m2\n")
        write_table(CEll_HRU_TABLE, file)
        
        # Write dynamic table
        file.write("Cell_ID\tArea_m2\tHRU\tpoly_area_m2\n")
        
        write_table(HRU_CELL_TABLE, file)
        
    print("gwflow file created successfully.")

def process_path_objects(path_objects, grids_point_path, gis_folder):
    outpath_object = {}

    for item in path_objects:
        path = item['path']
        name = item['name']
        Type = item['Type']

        if Type == 'raster':
            outpath = extract_raster_values_to_points(grids_point_path, path, f'{name}_shape.shp', gis_folder, name)
            outpath_object[name] = outpath
        elif Type == "shape":
            object = gpd.read_file(path)[[name, 'geometry']]

            if name in ['HRUS', 'Channel']:
                
                print(f'length of {name} array:', len(object[name]))
                print(f'Maximum value of {name} ID:', object[name].max())
                print(f'Unique {name} IDs:', len(object[name].unique()))
                
            if name=='HRUS':
                
                object['HRUs_Area_m2'] = object.geometry.area
                object['HRUs_Area_m2'] = object['HRUs_Area_m2'].astype(float).round(2)
                data = gpd.overlay(object[[name,'HRUs_Area_m2' ,'geometry']], grids_polygon[["Cell_ID", 'geometry']], how='intersection')
                print(f"Extracted values saved to: {name}_shape.pkl")
                outpath = os.path.join(gis_folder, f'{name}_shape.pkl')
                data[['Cell_ID','HRUs_Area_m2', name, 'geometry']].to_pickle(outpath)
            
            else:
                
                data = gpd.overlay(object[[name, 'geometry']], grids_polygon[["Cell_ID", 'geometry']], how='intersection')
                
                print(f"Extracted values saved to: {name}_shape.pkl")
                outpath = os.path.join(gis_folder, f'{name}_shape.pkl')
                data[['Cell_ID', name, 'geometry']].to_pickle(outpath)
            
            outpath_object[name] = outpath

    return outpath_object



In [6]:
HRUs = pd.read_pickle(os.path.join(gis_folder, f'HRUS_shape.pkl'))
HRUs

,Cell_ID,HRUs_Area_m2,HRUS,geometry
0,748,1378.847953,1510,"POLYGON ((4126310.627 188947.165, 4126310.627 ..."
1,748,51706.798233,1544,"MULTIPOLYGON (((4126231.856 188947.165, 412631..."
2,748,64116.429809,1545,"POLYGON ((4126074.313 188894.651, 4126126.827 ..."
3,749,1378.847953,1510,"POLYGON ((4126315.705 188947.165, 4126315.705 ..."
4,749,64805.853786,1511,"POLYGON ((4126363.141 188947.165, 4126363.141 ..."
...,...,...,...,...
55424,9400,130990.555525,14670,"POLYGON ((4105065.705 174926.063, 4105065.705 ..."
55425,9868,130990.555525,14670,"POLYGON ((4105199.980 174190.874, 4105315.705 ..."
55426,9869,130990.555525,14670,"POLYGON ((4105462.550 174190.874, 4105462.550 ..."
55427,10024,130990.555525,14670,"POLYGON ((4105199.980 173928.307, 4105199.980 ..."


In [8]:
LEVEL = "huc12"
RESOLUTION = 250
BASE_PATH = fr"/data/MyDataBase/SWATGenXAppData/"
NAMES = os.listdir("D:\MyDataBase\SWATplus_by_VPUID\huc12")
NAME = '40900010209'
CRS = 26990

# Pathways for local and output files
gwflow_target_path = os.path.join(BASE_PATH, f"SWATplus_by_VPUID/{LEVEL}/{NAME}/gwflow")
gwflow_data = os.path.join(gwflow_target_path, "gwflow.data")
gis_folder = os.path.join(gwflow_target_path, "gwflow_gis")
print('Target path:', gwflow_target_path)
print('environment:',gis_folder)
clear_directory(gis_folder)


grids_polygon_path = os.path.join(BASE_PATH, f"SWATplus_by_VPUID/{LEVEL}/{NAME}/MODFLOW_250/Grids_MODFLOW.pk1")
GW_RASTERS_dire = os.path.join(BASE_PATH, f"SWATplus_by_VPUID/{LEVEL}/{NAME}/MODFLOW_{RESOLUTION}/rasters_input/")
SWAT_shape_dire = os.path.join(BASE_PATH, f"SWATplus_by_VPUID/{LEVEL}/{NAME}/SWAT_MODEL/Watershed/Shapes/") 


thickrast_path = os.path.join(GW_RASTERS_dire, f"{NAME}_kriging_output_AQ_THK_1_{RESOLUTION}m.tif.tif")  # EPSG:26990
perm_path = os.path.join(GW_RASTERS_dire, f"{NAME}_kriging_output_H_COND_1_{RESOLUTION}m.tif.tif")  # EPSG:26990
SWL_path = os.path.join(GW_RASTERS_dire, f"{NAME}_kriging_output_SWL_{RESOLUTION}m.tif.tif")  # EPSG:26990
domain_path = os.path.join(GW_RASTERS_dire, f"{NAME}_raster.tif")  # EPSG:26990  
DEM_path = os.path.join(GW_RASTERS_dire, f"{NAME}_DEM_250m.tif.tif")  # EPSG:26990  
Error_path = os.path.join(GW_RASTERS_dire, f"{NAME}_kriging_stderr_H_COND_1_{RESOLUTION}m.tif.tif")  # EPSG:26990  
hrus_path = os.path.join(SWAT_shape_dire, 'hrus2.shp')  # EPSG:26990
streams_path = os.path.join(SWAT_shape_dire, "rivs1.shp")  # EPSG:26990
subbasins_path = os.path.join(SWAT_shape_dire, "subs1.shp")  # EPSG:26990
lakes_path = os.path.join(SWAT_shape_dire, "SWAT_plus_lakes.shp")  # EPSG:26990

bound_path, active_domain_path = defining_bound_and_active(subbasins_path, gis_folder)
grids_polygon , grids_point_path = read_grids (grids_polygon_path, CRS)


path_object = [
    { 'Type':'shape' , 'path': hrus_path, 'name':'HRUS'},
    { 'Type':'shape' ,'path': bound_path, 'name':'Bound'},
    { 'Type':'shape' ,'path': active_domain_path, 'name':'Active'},
    { 'Type':'shape' ,'path': streams_path, 'name':'Channel'},
    { 'Type':'shape' , 'path': subbasins_path,'name':'Subbasin'},
    { 'Type':'shape' ,'path': lakes_path,'name':'LakeId'},
    {'Type':'raster','path': thickrast_path, 'name':'Thick'},
    {'Type':'raster','path': perm_path, 'name':'HHC'},
    {'Type':'raster','path': SWL_path, 'name':'SWL'},
    {'Type':'raster', 'path': DEM_path,'name':'Elev'},
    {'Type':'raster', 'path': Error_path,'name':'Error'},
    
]




# Example usage
outpath_object = process_path_objects(path_object, grids_point_path, gis_folder)

Zone, ZONE_K_TABLE, outpath_object['Zone'] = creating_zones(outpath_object['HHC'])
outpath_object['grids_point'] = grids_point_path
outpath_object['grids_polygon'] = grids_polygon_path


Target path: /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40900010209/gwflow
environment: /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40900010209/gwflow\gwflow_gis
Generated shap saved to: bound_shape.shp
Generated shape saved to: active_domain_shape.shp
length of HRUS array: 14704
Maximum value of HRUS ID: 9999
Unique HRUS IDs: 14704
Extracted values saved to: HRUS_shape.pkl
Extracted values saved to: Bound_shape.pkl
Extracted values saved to: Active_shape.pkl
length of Channel array: 508
Maximum value of Channel ID: 512
Unique Channel IDs: 508
Extracted values saved to: Channel_shape.pkl
Extracted values saved to: Subbasin_shape.pkl
Extracted values saved to: LakeId_shape.pkl
Extracted values saved to: Thick_shape.shp
Extracted values saved to: HHC_shape.shp
Extracted values saved to: SWL_shape.shp
Extracted values saved to: Elev_shape.shp
Extracted values saved to: Error_shape.shp
first_quartile: 11.3512995
median: 19.3616757
third_quartile: 30.6978434999999

In [51]:

CEll_HRU_TABLE, HRU_CELL_TABLE, CELL_CHANNEL_TABLE = getting_TABLES (outpath_object)

grids_point , GRIDS_TABLE, GRIDS_TABLE_path, num_rows, num_cols  = getting_grid_table(outpath_object)

write_gwflow_file(gwflow_target_path, ZONE_K_TABLE, GRIDS_TABLE, CELL_CHANNEL_TABLE, HRU_CELL_TABLE,CEll_HRU_TABLE, num_rows, num_cols)


first_quartile: 11.3512995
median: 19.3616757
third_quartile: 30.697843499999998
Zones are created and saved: Zone_Shape.shp
Length of channels_cell array: 3107
Maximum number of Channels: 512
Length of CELL_HRUs array: 55429
Maximum number of HRUs: 14704
check: Number of grid cells loaded 13728
check: Total number of grids cell 13728
gwflow file created successfully.


In [52]:

os.chdir(fr"D:\MyDataBase\SWATplus_by_VPUID\huc12\{NAME}\gwflow")

!gwflow_input_files.exe


 Write out gwflow.input
 Write out gwflow.rivcells
 Write out gwflow.cellhru
 Write out gwflow.hrucell


forrtl: severe (408): fort: (10): Subscript #1 of the array HRU_CONNECTED has value 13729 which is greater than the upper bound of 13728

Image              PC                Routine            Line        Source             
gwflow_input_file  00007FF68D005961  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68CFE345C  MAIN__                    415  gwflow_input_files.f
gwflow_input_file  00007FF68D09D052  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68D09E329  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68D09E24E  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68D09E10E  Unknown               Unknown  Unknown
gwflow_input_file  00007FF68D09E399  Unknown               Unknown  Unknown
KERNEL32.DLL       00007FFB5BDE4DE0  Unknown               Unknown  Unknown
ntdll.dll          00007FFB5C0FED9B  Unknown               Unknown  Unknown


In [40]:
#### iNFORMATION i NEED

####  We have 4 tables:
#### 1- zone table (zone, k, sy, n)
#### 2- grid table (cell_Id, Thinck, Head, Zone, and nutrient&ET)
#### 3- 


,Cell_ID,Area_m2,HRUS,poly_area_m2
0,6364,62500.0,1,673.76
1,6520,62500.0,1,8978.17
2,6831,62500.0,1,4964.91
3,6832,62500.0,1,5661.69
4,6985,62500.0,1,5851.12
...,...,...,...,...
55424,9862,62500.0,14701,771.70
55425,9863,62500.0,14701,4743.69
55426,9863,62500.0,14702,7583.66
55427,9863,62500.0,14703,1378.85
